In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(4321)

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_val = x_train[50000:60000]
x_train = x_train[0:50000]

y_val = y_train[50000:60000]
y_train = y_train[0:50000]

x_train = x_train.astype(np.float32).reshape(-1,28,28,1) / 255.0
x_val = x_val.astype(np.float32).reshape(-1,28,28,1) / 255.0
x_test = x_test.astype(np.float32).reshape(-1,28,28,1) / 255.0

y_train = tf.one_hot(y_train, depth=10)
y_val = tf.one_hot(y_val, depth=10)
y_test = tf.one_hot(y_test, depth=10)
print(x_train.shape)
print(x_val.shape)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))


(50000, 28, 28, 1)
(10000, 28, 28, 1)


In [9]:
class ImageRecognitionCNN(tf.keras.Model):
    
    def __init__(self, num_classes, device='cpu:0', checkpoint_directory=None):
        ''' Define the parameterized layers used during forward-pass, the device
            where you would like to run the computation (GPU, TPU, CPU) on and the checkpoint
            directory.
            
            Args:
                num_classes: the number of labels in the network.
                device: string, 'cpu:n' or 'gpu:n' (n can vary). Default, 'cpu:0'.
                checkpoint_directory: the directory where you would like to save or 
                                      restore a model.
        ''' 
        super(ImageRecognitionCNN, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(64, 3, padding='same', activation=None)
        self.conv2 = tf.keras.layers.Conv2D(64, 3, padding='same', activation=None)
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv3 = tf.keras.layers.Conv2D(64, 3, padding='same', activation=None)
        self.conv4 = tf.keras.layers.Conv2D(64, 3, padding='same', activation=None)
        self.conv8 = tf.keras.layers.Conv2D(num_classes, 1, padding='same', activation=None)
        self.bn1=tf.keras.layers.BatchNormalization()
        self.bn2=tf.keras.layers.BatchNormalization()
        self.bn3=tf.keras.layers.BatchNormalization()
        self.bn4=tf.keras.layers.BatchNormalization()
        
        # Define the device 
        self.device = device
        
        # Define the checkpoint directory
        self.checkpoint_directory = checkpoint_directory
        self.acc = tf.keras.metrics.Accuracy()


    def predict(self, images, training):
        x = self.conv1(images)
        x = tf.nn.relu(x)
        x= self.bn1(x,training=training)
        x = self.pool1(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x= self.bn2(x,training=training)
        x = self.pool1(x)
        x = self.conv3(x)
        x = tf.nn.relu(x)
        x= self.bn3(x,training=training)
        x = self.pool1(x)
        x = self.conv4(x)
        x = tf.nn.relu(x)
        x= self.bn4(x,training=training)
        x = self.pool1(x)
        x = self.conv8(x)
        x = tf.reshape(x, (-1, 1, 10))
        return x


    def loss_fn(self, images, target, training):
        """ Defines the loss function used during 
            training.         
        """
        preds = self.predict(images, training)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=target, logits=preds)
        return loss


    def grads_fn(self, images, target, training):

        with tf.GradientTape() as tape:
            loss = self.loss_fn(images, target, training)
        return tape.gradient(loss, self.trainable_variables)
    
    def restore_model(self):
      
        with tf.device(self.device):
            dummy_input = tf.constant(tf.zeros((1,48,48,1)))
            dummy_pred = self.predict(dummy_input, training=False)
            saver = tf.Saver(self.variables)
            saver.restore(tf.train.latest_checkpoint
                          (self.checkpoint_directory))
    
    def save_model(self, global_step=0):
        """ Function to save trained model.
        """
        tf.Saver(self.variables).save(self.checkpoint_directory, 
                                       global_step=global_step)   

    def compute_accuracy_2(self, images, targets):
        """ Compute the accuracy on the input data.
        """
        with tf.device(self.device):
            
            logits = self.predict(images, training=False)
            
            logits = tf.nn.softmax(logits)
            logits = tf.reshape(logits, [-1, 10])
            targets = tf.reshape(targets, [-1,10])
            preds = tf.argmax(logits, axis=1)
            goal = tf.argmax(targets, axis=1)
            self.acc.update_state(goal, preds)
            result = self.acc.result().numpy()
        return result

  
    def fit_fc(self, training_data, eval_data, optimizer, num_epochs=500, 
            early_stopping_rounds=10, verbose=10, train_from_scratch=False):
        """ Function to train the model, using the selected optimizer and
            for the desired number of epochs. You can either train from scratch
            or load the latest model trained. Early stopping is used in order to
            mitigate the risk of overfitting the network.
            
            Args:
                training_data: the data you would like to train the model on.
                                Must be in the tf.data.Dataset format.
                eval_data: the data you would like to evaluate the model on.
                            Must be in the tf.data.Dataset format.
                optimizer: the optimizer used during training.
                num_epochs: the maximum number of iterations you would like to 
                            train the model.
                early_stopping_rounds: stop training if the loss on the eval 
                                       dataset does not decrease after n epochs.
                verbose: int. Specify how often to print the loss value of the network.
                train_from_scratch: boolean. Whether to initialize variables of the
                                    the last trained model or initialize them
                                    randomly.
        """ 
    
        if train_from_scratch==False:
            self.restore_model()
        
  
        best_loss = 999
        
        train_loss = tf.keras.metrics.Mean('train_loss')
        eval_loss = tf.keras.metrics.Mean('eval_loss')
        acc_train = tf.keras.metrics.Mean('train_acc')
        acc_val = tf.keras.metrics.Mean('val_acc')
        
        self.history = {}
        self.history['train_loss'] = []
        self.history['eval_loss'] = []
        self.history['train_acc'] = []
        self.history['val_acc'] = []
        
        with tf.device(self.device):
            for i in range(num_epochs):
                # Training with gradient descent
                training_data1 = training_data.shuffle(buffer_size=1024).batch(125)
                for step, (images, target) in enumerate(training_data1):
                    grads = self.grads_fn(images, target, True)
                    optimizer.apply_gradients(zip(grads, self.trainable_variables))
                
              
                    
                # Compute the loss on the training data after one epoch
                training_data1=training_data.shuffle(buffer_size=1024).batch(125)
                for step, (images, target) in enumerate(training_data1):
                    loss = self.loss_fn(images, target, False)
                    accuracy = self.compute_accuracy_2(images,target)
                    acc_train(accuracy)
                    train_loss(loss)
                self.history['train_loss'].append(train_loss.result().numpy())
                self.history['train_acc'].append(acc_train.result().numpy())
                # Reset metrics
                train_loss.reset_states()
                acc_train.reset_states()
                
                # Compute the loss on the eval data after one epoch
                eval_data1=eval_data.shuffle(buffer_size=1024).batch(125)
                for step, (images, target) in enumerate(eval_data1):
                    loss = self.loss_fn(images, target, False)
                    accuracy = self.compute_accuracy_2(images,target)
                    acc_val(accuracy)
                    eval_loss(loss)
                self.history['eval_loss'].append(eval_loss.result().numpy())
                self.history['val_acc'].append(acc_val.result().numpy())
                # Reset metrics
                eval_loss.reset_states()
                acc_val.reset_states()
                
                # Print train and eval losses
                if (i==0) or ((i+1)%verbose==0):
                    print('Train loss at epoch %d: ' %(i+1), self.history['train_loss'][-1])
                    print('Train Acc at epoch %d: ' %(i+1), self.history['train_acc'][-1])
                    
                    print('Eval loss at epoch %d: ' %(i+1), self.history['eval_loss'][-1])
                    print('Eval Acc at epoch %d: ' %(i+1), self.history['val_acc'][-1])

    def test(self, test_data):
        test_loss = tf.keras.metrics.Mean('test_loss')
        acc_test = tf.keras.metrics.Mean('test_acc')

        test_data=test_data.shuffle(buffer_size=1024).batch(125)
        for step, (images, target) in enumerate(test_data):
            loss = self.loss_fn(images, target, False)
            accuracy = self.compute_accuracy_2(images,target)
            acc_test(accuracy)
            test_loss(loss)
        return test_loss.result(),acc_test.result()

In [12]:
# Specify the path where you want to save/restore the trained variables.
checkpoint_directory = 'models_checkpoints/fashion_mnist/'

# Use the GPU if available.
device = 'cpu'
#device = 'gpu:0'

# Define optimizer.
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4)

# Instantiate model. This doesn't initialize the variables yet.
model = ImageRecognitionCNN(num_classes=10, device=device, 
                              checkpoint_directory=checkpoint_directory)


In [14]:
# Train model

model.fit_fc(train_dataset, val_dataset, optimizer, num_epochs=10, 
          early_stopping_rounds=2, verbose=2, train_from_scratch=True)

Train loss at epoch 1:  1.3529146
Train Acc at epoch 1:  0.52461064
Eval loss at epoch 1:  1.3756595
Eval Acc at epoch 1:  0.52007884
Train loss at epoch 2:  0.32412642
Train Acc at epoch 2:  0.618018
Eval loss at epoch 2:  0.36776605
Eval Acc at epoch 2:  0.69240016
Train loss at epoch 4:  0.23808286
Train Acc at epoch 4:  0.7834034
Eval loss at epoch 4:  0.3103498
Eval Acc at epoch 4:  0.8005363
Train loss at epoch 6:  0.19157645
Train Acc at epoch 6:  0.83322626
Eval loss at epoch 6:  0.2858305
Eval Acc at epoch 6:  0.8415104
Train loss at epoch 8:  0.16623612
Train Acc at epoch 8:  0.85957724
Eval loss at epoch 8:  0.2880917
Eval Acc at epoch 8:  0.86453754
Train loss at epoch 10:  0.13460858
Train Acc at epoch 10:  0.87663347
Eval loss at epoch 10:  0.29067996
Eval Acc at epoch 10:  0.8803026


In [15]:
loss, acc = model.test(test_dataset)
print("Loss = ", float(loss))
print("Accuracy = ", float(acc)*100)


Loss =  0.3099122643470764
Accuracy =  88.05135488510132
